1. 100일치 네이버 영화 평점 데이터 크롤링 하기
 - 제목, 스코어
2. 100일치 데이터를 mongodb에 insert

3. insert된 내용을 토대로 데이터프레임을 생성하고, 시각화 하기

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
data = requests.get('https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20200719',headers=headers)

In [3]:
soup = BeautifulSoup(data.text,'html.parser')
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="me2:image">
<meta content="네이버영화 " property="me2:post_tag">
<meta content="네이버영화" property="me2:category1"/>
<meta content="" property="me2:category2"/>
<meta content="랭킹 : 네이버 영화" property="og:title"/>
<meta content="영화, 영화인, 예매, 박스오피스 랭킹 정보 제공" property="og:description"/>
<meta content="article" property="og:type"/>
<meta content="https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&amp;date=20200719" property="og:url"/>
<meta content="http://static.naver.net/m/movie/icons/OG_270_270.png" property="og:image"/><!-- http://static.naver.net/m/movie/im/navermovie.jpg -->
<meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="og:article:thumbnailUrl"/>
<meta content="네이버 영화" property="og:article:author"/>


### css 선택자(selector)를 활용한 크롤링
- select() 활용하기

In [7]:
movies = soup.select('#old_content > table > tbody > tr')

In [8]:
len(movies) # 필요없는 요소까지 모두 가져와서 영화 50개보다 더 많을 것이다.

56

확인해보니 영화 이름은 a tag가 있음

In [10]:
# movie : 수집한 tr 태그
for movie in movies:
    a_tag = movie.select_one('td.title > div > a')
    
    # tr에 a_tag가 None이 아니면 ( 있으면)
    if a_tag is not None:
        rank = int(movie.select_one('td:nth-child(1) > img').get('alt'))
        title = a_tag.text
        star  = movie.select_one('td.point').text
        print(rank, title, star)

1 그린 북 9.59
2 가버나움 9.59
3 베일리 어게인 9.53
4 원더 9.49
5 포드 V 페라리 9.49
6 아일라 9.49
7 주전장 9.48
8 당갈 9.47
9 쇼생크 탈출 9.44
10 터미네이터 2:오리지널 9.44
11 보헤미안 랩소디 9.42
12 덕구 9.42
13 월-E 9.41
14 나 홀로 집에 9.41
15 라이언 일병 구하기 9.40
16 사운드 오브 뮤직 9.40
17 살인의 추억 9.40
18 빽 투 더 퓨쳐 9.40
19 매트릭스 9.40
20 인생은 아름다워 9.40
21 헬프 9.40
22 아이즈 온 미 : 더 무비 9.40
23 위대한 쇼맨 9.40
24 포레스트 검프 9.40
25 클래식 9.39
26 글래디에이터 9.39
27 센과 치히로의 행방불명 9.39
28 토이 스토리 3 9.38
29 타이타닉 9.38
30 알라딘 9.38
31 어벤져스: 엔드게임 9.38
32 안녕 베일리 9.37
33 죽은 시인의 사회 9.37
34 레옹 9.37
35 집으로... 9.37
36 헌터 킬러 9.37
37 반지의 제왕: 왕의 귀환 9.37
38 동주 9.37
39 아이 캔 스피크 9.37
40 캐스트 어웨이 9.37
41 히든 피겨스 9.36
42 굿바이 마이 프랜드 9.36
43 여인의 향기 9.36
44 굿 윌 헌팅 9.36
45 서유기 2 - 선리기연 9.36
46 클레멘타인 9.35
47 주토피아 9.35
48 쉰들러 리스트 9.35
49 달링 9.35
50 모노노케 히메 9.35


In [11]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.movie 

for movie in movies:
    a_tag = movie.select_one('td.title > div > a')
    
    if a_tag is not None:
        rank = int(movie.select_one('td:nth-child(1) > img').get('alt'))
        title = a_tag.text
        star  = movie.select_one('td.point').text
        
        movie_doc = {
            'rank':rank,
            'title': title,
            'star':star
            
        }
        db.movie.insert_one(movie_doc)